In [1]:
%matplotlib notebook

In [3]:
import functions
import plot_handler

In [ ]:
import pandas as pd

# Plots for Acending Phase

In [ ]:
df_asc = pd.read_csv('./data_csv.csv',
                        names=['time',
                               'P_in', 'P_out',
                               'T_in', 'T_out',
                               'Hum_in', 'Hum_out',
                               'CO2_V1', 'CO2_V2',
                               'O3_WE', 'O3_AE',
                               'Altitude'],
                      header=0,
                      usecols=[i for i in range(0,12)])

print(df_asc)

## Plots refreshed by clicking

In [ ]:
df_asc['O3_ppm'] = df_asc.apply(functions.O3Concentration, axis=1)


In [ ]:
df_asc['CO2_C'] = df_asc.apply(functions.CO2Concentration, axis=1)


In [ ]:
df_asc['flags'] = df_asc.apply(lambda x: 1 if (x["valve_1"]==1 and x["valve_2"]==0) else 0, axis=1)
df_asc['Flowrate'] = functions.flowrate(df_asc.mask(lambda x: x['flags']!=1))


## Plots automatically refreshed

In [ ]:
df_asc['Flowrate'] = functions.flowrate(df_asc.mask(lambda x: x['flags']!=1))

# Plots for Floating Phase

In [ ]:
df_float = pd.read_csv('./data_csv.csv',
                        names=['time',
                               'P_in', 'P_out',
                               'T_in', 'T_out',
                               'Hum_in', 'Hum_out',
                               'CO2_V1', 'CO2_V2',
                               'O3_WE', 'O3_AE',
                               'Altitude'],
                      header=0,
                      usecols=[i for i in range(0,12)])

print(df_float)

## Plots refreshed by clicking

In [ ]:
df_float['O3_ppm'] = df_float.apply(functions.O3Concentration, axis=1)


In [ ]:
df_float['CO2_C'] = df_float.apply(functions.CO2Concentration, axis=1)


## Plots automatically refreshed

In [ ]:
df_float['Flowrate'] = functions.flowrate(df_float.mask(lambda x: x['flags']!=1))